In [ ]:
# !pip install fvcore

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61443 sha256=b095a73daa309a24299305a486b8f4a5d18944a49e1d6530fc571f34db7366a1
  Stored in directory: /home/cysong/.cache/pip/wheels/83/42/02/66178d16e5c44dc26d309931834956baeda371956e86fbd876
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31570 sha256=52aa4bb8be1192ae751b418bcb72c9513157dcb3735cb18a53360eb766ebfb6e
  Stored in directory: /home/cysong/.cache/pip/wheels/c1/13/6d/441d8f2af76ee6d2a3e67eebb1d0c556fefcee0a8b32266a8e
Successfully built fvcore iopath

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
# !pip install av

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 17.5 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
import logging
logging.getLogger("pytorchvideo").setLevel(logging.CRITICAL)

import torch
from torch import nn
import numpy as np
from tqdm import tqdm
import os
import torchvision.utils as vutils

# Load X3D model
model_name = 'x3d_s'
model = torch.hub.load('facebookresearch/pytorchvideo', model_name, pretrained=True)
device = "cuda:0"
feature_model = model.eval().to(device)
del feature_model.blocks[-1]

Using cache found in /home/cysong/.cache/torch/hub/facebookresearch_pytorchvideo_main


In [ ]:
import logging
logging.getLogger("pytorchvideo").setLevel(logging.CRITICAL)

import torch
from torch import nn
import numpy as np
from tqdm import tqdm
import os
import torchvision.utils as vutils

# Load X3D model
model_name = 'x3d_s'
model = torch.hub.load('facebookresearch/pytorchvideo', model_name, pretrained=True)
device = "cuda:0"
feature_model = model.eval().to(device)
del feature_model.blocks[-1]

# Load deweathering model
def load_deweather_model(device):
    from .model.ESDNet import ESDNet
    deweather_model = ESDNet(
        en_feature_num=48,
        en_inter_num=32,
        de_feature_num=64,
        de_inter_num=32,
        sam_number=1
    ).to(device)

    load_path = 'weight/deweathering_model.pth'
    try:
        ckpt = torch.load(load_path, map_location=device)
        state_dict = ckpt if load_path.endswith('.pth') else ckpt['state_dict']
        deweather_model.load_state_dict(state_dict)
        print("Deweathering model loaded from", load_path)
    except Exception as e:
        print("Failed to load deweathering model checkpoint:", e)

    deweather_model.eval()
    return deweather_model


In [9]:
deweather_model = load_deweather_model(device).to(device)

ImportError: attempted relative import with no known parent package

In [ ]:
!pip install performer_pytorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 103.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 115.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvji

In [ ]:
from .model.classifier import Model

classifier = Model(ff_mult=1, dims=(32, 32), depths=(1, 1))
ckpt_path = "weight/De_final_model.pth"
classifier.load_state_dict(torch.load(ckpt_path, map_location=device))
classifier.to(device)

Model(
  (stages): ModuleList(
    (0): ConvBlock(
      (norm1): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
      (conv): DECOUPLED(
        (norm2d): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (norm1d): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2d): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16)
        (conv1d): Conv1d(32, 32, kernel_size=(3,), stride=(1,), padding=(1,), groups=16)
      )
      (ff): Sequential(
        (0): Linear(in_features=32, out_features=32, bias=True)
        (1): GELU(approximate='none')
        (2): Dropout(p=0.2, inplace=False)
        (3): Linear(in_features=32, out_features=32, bias=True)
        (4): GELU(approximate='none')
      )
    )
    (1): Sequential(
      (0): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
      (1): Linear(in_features=

In [ ]:
import logging
import os
import time
import cv2
import torch
import numpy as np
from torch import nn
from PIL import Image
from tqdm import tqdm
from collections import deque
from torchvision.transforms import ToTensor, Normalize, Resize, Compose

logging.getLogger("pytorchvideo").setLevel(logging.CRITICAL)

# Load X3D model
model_name = 'x3d_s'
model = torch.hub.load('facebookresearch/pytorchvideo', model_name, pretrained=True)
device = "cuda:0"
feature_model = model.eval().to(device)
del feature_model.blocks[-1]

# Load deweathering model
def load_deweather_model(device):
    from model.ESDNet import ESDNet
    model = ESDNet(48, 32, 64, 32, 1).to(device)
    path = 'weight/deweathering_model.pth'
    try:
        ckpt = torch.load(path, map_location=device)
        state_dict = ckpt if path.endswith('.pth') else ckpt['state_dict']
        model.load_state_dict(state_dict)
        print("Deweathering model loaded from", path)
    except Exception as e:
        print("Failed to load deweathering model checkpoint:", e)
    return model.eval()

deweather_model = load_deweather_model(device)

# Configuration
CLIP_LEN = 13
IMG_SIZE = 160
MEAN = [0.45] * 3
STD = [0.225] * 3
MAX_FRAMES = 200
THRESHOLD = 0.60
ALERT_FRAMES = 20

transform_frame = Compose([
    Resize((IMG_SIZE, IMG_SIZE)),
    ToTensor()
])
normalize = Normalize(MEAN, STD)

frame_buffer = deque(maxlen=CLIP_LEN)
anomaly_streak = 0

cap = cv2.VideoCapture("/mnt/d/Capstone/data/UCF_snowsynth/Videos/Fighting/Fighting007_x264.mp4")
assert cap.isOpened(), "Failed to open video."

frame_count = 0
start_time = time.time()

while frame_count < MAX_FRAMES:
    ret, frame = cap.read()
    if not ret:
        print("Video ended or error occurred.")
        break

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pil = Image.fromarray(rgb)
    tensor = transform_frame(pil).unsqueeze(0).to(device)

    with torch.no_grad():
        output = deweather_model(tensor)
        clean = output[0] if isinstance(output, tuple) else output

    image = clean.squeeze(0).cpu().numpy().transpose(1, 2, 0)
    image = np.clip(image * 255.0, 0, 255).astype(np.uint8)
    image_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    frame_buffer.append(clean.squeeze(0))

    label = "Collecting..."
    score = None

    if len(frame_buffer) == CLIP_LEN:
        clip = torch.stack(list(frame_buffer), dim=0).permute(1, 0, 2, 3)
        clip = torch.stack([normalize(clip[:, t]) for t in range(CLIP_LEN)], dim=1)
        clip = clip.unsqueeze(0).to(device)

        with torch.no_grad():
            feat = feature_model(clip)
            from model.classifier import Model
            classifier = Model(ff_mult=1, dims=(32, 32), depths=(1, 1)).to(device)
            checkpoint = torch.load("weight/De_final_model.pth", map_location=device)
            classifier.load_state_dict(checkpoint)
            classifier.eval()
            pred, _ = classifier(feat)
            score = torch.sigmoid(pred).item()
            label = f"A: {score:.2f}"

        anomaly_streak = anomaly_streak + 1 if score >= THRESHOLD else 0

    elapsed = time.time() - start_time
    fps = frame_count / elapsed if elapsed > 0 else 0
    text = f"FPS: {fps:.2f} | {label}"

    cv2.putText(image_bgr, text, (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 255, 255), 1)

    if anomaly_streak >= ALERT_FRAMES:
        h, w = image_bgr.shape[:2]
        cv2.rectangle(image_bgr, (0, 0), (w - 1, h - 1), (0, 0, 255), 3)

    cv2.imshow("Anomaly Detection", image_bgr)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    frame_count += 1

cap.release()
cv2.destroyAllWindows()
print("Processing complete.")

Output hidden; open in https://colab.research.google.com to view.